In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
from config.utils import *

In [4]:
DATA_ROOT

'c:\\Users\\BROliveiWa12\\OneDrive - NESTLE\\Documents\\Estudos\\Engenharia de Dados\\economic-indicators-pipeline\\notebooks'

In [4]:
os.path.join(DATA_ROOT, "bronze")

'data/bronze'

In [5]:
INDICADORES = {
    "IPCA": {
        "NOME": "Índice Nacional de Preços ao Consumidor Amplo (IPCA)",
        "CATEGORIA": "INFLACAO",
        "FONTE": API["IBGE"]["FONTE"],
        "URL": str(f"{API['IBGE']['URL']}/{PARAMETROS['IBGE']['IPCA']['AGREGADO']}/periodos/all/variaveis/{PARAMETROS['IBGE']['IPCA']['VARIAVEL']}?localidades=N1[all]")
    },
    "SELIC_META": {
        "NOME": "Taxa Selic Meta",
        "CATEGORIA": "JUROS",
        "FONTE": API["BACEN"]["FONTE"],
        "URL": str(f"{API['BACEN']['URL']}{PARAMETROS['BACEN']['SELIC_META']}/dados?formato=json&dataInicial={DATA_INICIAL}&dataFinal={DATA_FINAL}")
    },
    "IBC_BR": {
         "NOME": "Índice de Atividade Econômica do Banco Central (IBC-Br)",
        "CATEGORIA": "atividade_economica",
        "FONTE": API["BACEN"]["FONTE"],
        "URL": str(f"{API['BACEN']['URL']}{PARAMETROS['BACEN']['IBC_BR']}/dados?formato=json&dataInicial={DATA_INICIAL}&dataFinal={DATA_FINAL}")
    },
    "DESEMPREGO": {
        "NOME": "Taxa de Desemprego",
        "CATEGORIA": "MERCADO_TRABALHO",
        "FONTE": API['IBGE']['FONTE'],
        "URL": str(f"{API['IBGE']}/{PARAMETROS['IBGE']['DESEMPREGO']['AGREGADO']}/periodos/all/variaveis/{PARAMETROS['IBGE']['DESEMPREGO']['VARIAVEL']}?localidades=N1[all]")
    },
    "INADIMPLENCIA_PF": {
        "NOME": "Inadimplência Pessoa Física",
        "CATEGORIA": "CREDITO_CONSUMO",
        "FONTE": "BACEN",
        "URL": str(f"{API['BACEN']['URL']}{PARAMETROS['BACEN']['INADIMPLENCIA_PF']}/dados?formato=json&dataInicial={DATA_INICIAL}&dataFinal={DATA_FINAL}")
    }
}


In [6]:
def obter_dados_api(indicador:dict, timeout: int = 30) -> dict:
    """
    Realiza uma requisição GET a uma API pública e retorna os dados em formato JSON.

    Args:
        url (str): URL completa do endpoint da API.
        timeout (int): Tempo máximo de espera pela resposta, em segundos (padrão: 30s).

    Returns:
        dict ou list: Resposta da API convertida para objeto Python.

    Raises:
        Exception: Caso ocorra erro de conexão ou resposta inválida.
    """
    try:
        logger.info(f"Realizando requisição da API: {indicador['NOME']}")
        resposta = requests.get(indicador['URL'], timeout=timeout)
        resposta.raise_for_status()  
        logger.success(f"Requisição para {indicador['NOME']} realizada com sucesso - Status: {resposta.status_code}")
        return resposta.json()
        

    except requests.exceptions.RequestException as erro:
        logger.error(f"Falha ao acessar a URL: {indicador['URL']}")
        raise erro

In [7]:
teste = obter_dados_api(INDICADORES['IPCA'])

2025-06-30 00:16:48.628 | INFO     | __main__:obter_dados_api:16 - Realizando requisição da API: Índice Nacional de Preços ao Consumidor Amplo (IPCA)
2025-06-30 00:16:48.812 | SUCCESS  | __main__:obter_dados_api:19 - Requisição para Índice Nacional de Preços ao Consumidor Amplo (IPCA) realizada com sucesso - Status: 200


In [8]:
def salvar_dados_bronze(dados: dict, nome_indicador: str) -> None:
    """
    Salva os dados brutos da API na camada bronze com nome padronizado.

    Args:
        dados (dict): Dados retornados da API.
        nome_indicador (str): Nome para salvar os metadados do indicador.

    Returns:
        None
    """
    try:
        nome_arquivo = f"{nome_indicador.lower()}_{dt.datetime.today().strftime('%Y%m%d')}.json"
        caminho_arquivo = os.path.join(PATHS["BRONZE_LAYER"],nome_arquivo)
        print(caminho_arquivo)

        with open(caminho_arquivo, "w", encoding="utf-8") as f:
            json.dump(dados, fp=f, ensure_ascii=False, indent=4)

        logger.info(f"Dados salvos com sucesso: {caminho_arquivo}")

    except Exception as e:
        logger.error(f"Erro ao salvar os dados na camada bronze: {str(e)}")
        raise e

In [9]:
os.path.join(PATHS["BRONZE_LAYER"],'teste.json')

'data/bronze\\teste.json'

In [13]:
teste[0]['resultados'][0]['series'][0]['serie']

{'202001': '0.21',
 '202002': '0.25',
 '202003': '0.07',
 '202004': '-0.31',
 '202005': '-0.38',
 '202006': '0.26',
 '202007': '0.36',
 '202008': '0.24',
 '202009': '0.64',
 '202010': '0.86',
 '202011': '0.89',
 '202012': '1.35',
 '202101': '0.25',
 '202102': '0.86',
 '202103': '0.93',
 '202104': '0.31',
 '202105': '0.83',
 '202106': '0.53',
 '202107': '0.96',
 '202108': '0.87',
 '202109': '1.16',
 '202110': '1.25',
 '202111': '0.95',
 '202112': '0.73',
 '202201': '0.54',
 '202202': '1.01',
 '202203': '1.62',
 '202204': '1.06',
 '202205': '0.47',
 '202206': '0.67',
 '202207': '-0.68',
 '202208': '-0.36',
 '202209': '-0.29',
 '202210': '0.59',
 '202211': '0.41',
 '202212': '0.62',
 '202301': '0.53',
 '202302': '0.84',
 '202303': '0.71',
 '202304': '0.61',
 '202305': '0.23',
 '202306': '-0.08',
 '202307': '0.12',
 '202308': '0.23',
 '202309': '0.26',
 '202310': '0.24',
 '202311': '0.28',
 '202312': '0.56',
 '202401': '0.42',
 '202402': '0.83',
 '202403': '0.16',
 '202404': '0.38',
 '2024

In [ ]:
json.dump(teste,fp=('teste'))

TypeError: dump() missing 1 required positional argument: 'fp'

In [10]:
salvar_dados_bronze(teste, 'ipca')

2025-06-30 00:05:20.849 | ERROR    | __main__:salvar_dados_bronze:23 - Erro ao salvar os dados na camada bronze: [Errno 2] No such file or directory: 'c:\\Users\\BROliveiWa12\\OneDrive - NESTLE\\Documents\\Estudos\\Engenharia de Dados\\economic-indicators-pipeline\\notebooks\\data\\bronze\\ipca_20250630.json'


c:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\notebooks\data\bronze\ipca_20250630.json


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\BROliveiWa12\\OneDrive - NESTLE\\Documents\\Estudos\\Engenharia de Dados\\economic-indicators-pipeline\\notebooks\\data\\bronze\\ipca_20250630.json'

In [35]:

resp = req.get(INDICADORES['IPCA']['URL'])
catalogo = resp.json()

In [36]:
catalogo

[{'id': '63',
  'variavel': 'IPCA - Variação mensal',
  'unidade': '%',
  'resultados': [{'classificacoes': [{'id': '315',
      'nome': 'Geral, grupo, subgrupo, item e subitem',
      'categoria': {'7169': 'Índice geral'}}],
    'series': [{'localidade': {'id': '1',
       'nivel': {'id': 'N1', 'nome': 'Brasil'},
       'nome': 'Brasil'},
      'serie': {'202001': '0.21',
       '202002': '0.25',
       '202003': '0.07',
       '202004': '-0.31',
       '202005': '-0.38',
       '202006': '0.26',
       '202007': '0.36',
       '202008': '0.24',
       '202009': '0.64',
       '202010': '0.86',
       '202011': '0.89',
       '202012': '1.35',
       '202101': '0.25',
       '202102': '0.86',
       '202103': '0.93',
       '202104': '0.31',
       '202105': '0.83',
       '202106': '0.53',
       '202107': '0.96',
       '202108': '0.87',
       '202109': '1.16',
       '202110': '1.25',
       '202111': '0.95',
       '202112': '0.73',
       '202201': '0.54',
       '202202': '1.01

In [23]:
response = req.get("https://servicodados.ibge.gov.br/api/v3/agregados")
data = response.json()

In [28]:
pd.DataFrame(data).head(50)

,id,nome,agregados
0,D5,Áreas Urbanizadas,"[{'id': '8418', 'nome': 'Áreas urbanizadas, Lo..."
1,CL,Cadastro Central de Empresas,"[{'id': '1685', 'nome': 'Unidades locais, empr..."
2,CA,Censo Agropecuário,"[{'id': '1005', 'nome': 'Número de estabelecim..."
3,ME,"Censo Comum do Mercosul, Bolívia e Chile","[{'id': '1221', 'nome': 'População residente, ..."
4,CD,Censo Demográfico,"[{'id': '10049', 'nome': 'Moradores indígenas ..."
5,CM,Contagem da População,"[{'id': '305', 'nome': 'População residente em..."
6,DU,Contas de ecossistemas: o uso da terra nos bio...,"[{'id': '7016', 'nome': 'Contas de extensão do..."
7,DT,Contas de Espécies Ameaçadas,"[{'id': '7392', 'nome': 'Número de espécies da..."
8,C4,Contas Econômicas Ambientais da Água,"[{'id': '6834', 'nome': 'Total de Recursos Híd..."
9,DR,Contas Econômicas Ambientais da Terra: Contabi...,"[{'id': '7319', 'nome': 'Estoque, por classes ..."


In [ ]:
pd.DataFrame(data[[-]])

TypeError: list indices must be integers or slices, not list

In [ ]:
import json

In [ ]:
import requests
import pandas as pd



BASE_URL_CAMBIO = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}"

def get_cotacao(codigo_serie):
    url = BASE_URL_CAMBIO.format(codigo_serie, START_DATE, END_DATE)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print(f"Erro ao buscar série {codigo_serie}: {response.status_code}")
        return None

if __name__ == "__main__":
    for nome, codigo in COTACOES_CAMBIO.items():
        print(f"Buscando cotação {nome}...")
        df_cotacao = get_cotacao(codigo)
        if df_cotacao is not None:
            print(df_cotacao.head())
            # aqui você pode salvar o df, ou seguir com o processamento


Buscando cotação usd_brl...
Erro ao buscar série 1: 400
Buscando cotação eur_brl...
Erro ao buscar série 21619: 400
Buscando cotação gbp_brl...
Erro ao buscar série 21623: 400
Buscando cotação jpy_brl...
Erro ao buscar série 21620: 400
Buscando cotação ars_brl...
Erro ao buscar série 21622: 400
Buscando cotação chf_brl...
Erro ao buscar série 21621: 400
Buscando cotação cny_brl...
Erro ao buscar série 21627: 400


In [ ]:
from datetime import datetime

In [ ]:
response = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.21619/dados?formato=json&dataInicial=29/06/2015")
response.json()

[{'data': '29/06/2015', 'valor': '3.5094000'},
 {'data': '30/06/2015', 'valor': '3.4603000'},
 {'data': '01/07/2015', 'valor': '3.4569000'},
 {'data': '02/07/2015', 'valor': '3.4577000'},
 {'data': '03/07/2015', 'valor': '3.4701000'},
 {'data': '06/07/2015', 'valor': '3.4813000'},
 {'data': '07/07/2015', 'valor': '3.4840000'},
 {'data': '08/07/2015', 'valor': '3.5743000'},
 {'data': '09/07/2015', 'valor': '3.5460000'},
 {'data': '10/07/2015', 'valor': '3.5568000'},
 {'data': '13/07/2015', 'valor': '3.4823000'},
 {'data': '14/07/2015', 'valor': '3.4403000'},
 {'data': '15/07/2015', 'valor': '3.4575000'},
 {'data': '16/07/2015', 'valor': '3.4192000'},
 {'data': '17/07/2015', 'valor': '3.4535000'},
 {'data': '20/07/2015', 'valor': '3.4882000'},
 {'data': '21/07/2015', 'valor': '3.4762000'},
 {'data': '22/07/2015', 'valor': '3.4934000'},
 {'data': '23/07/2015', 'valor': '3.6053000'},
 {'data': '24/07/2015', 'valor': '3.6503000'},
 {'data': '27/07/2015', 'valor': '3.7295000'},
 {'data': '28